# Metadata

**L1 Taxonomy** - SQL in Python

**L2 Taxonomy** - SQLite

**Subtopic** - Python SQLite Transaction Management

**Use Case** - Create a Python script to implement optimistic concurrency control in SQLite, detecting and resolving conflicts when multiple transactions attempt to update the same data.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

I want to design a Python module that simulates Optimistic Concurrency Control(OCC) using SQLite. It should be able to accept a list of thread constructor calls as input and run each transaction concurrently using threads.

For Setup:

- Start with a shared, in-memory SQLite DB with predefined schema.
```
CREATE TABLE inventory (
    id INTEGER PRIMARY KEY,
    item_name TEXT,
    quantity INTEGER,
    version INTEGER
);
```
- Add a data entry to the TABLE.
```
INSERT INTO inventory (id, item_name, quantity, version)
VALUES (1, 'Health Potion', 10, 1);

- This uses SQLite’s shared in-memory URI:

```
DB_URI = "file:occdb?mode=memory&cache=shared"
```
**Input Format**

A list of Python threading.Thread() definitions, where each target is a run_transaction(name: str, delay: float) function. run_transaction is a function that will simulate a database transaction that reads an item's quantity and version, waits for a delay, and attempts to update it using optimistic concurrency—committing only if the version hasn't changed, otherwise rolling back and reporting a conflict..

**Output Format**

A list of result dictionaries having:
- transaction: name of transaction
- status: committed/conflict(add reason in case of conflict)
- final_quantity: updated value
- final_version: latest version

**Examples**

```
Input:
[
    threading.Thread(target=run_transaction, args=("Transaction A", 0.3)),
    threading.Thread(target=run_transaction, args=("Transaction B", 0.5)),
]

Output:
[
    {
        "transaction": "Transaction A",
        "status": "committed",
        "final_quantity": 9,
        "final_version": 2
    },
    {
        "transaction": "Transaction B",
        "status": "conflict",
        "reason": "Expected version=1, found different."
    }
]

```


# Requirements

**Explicit and Implicit Points**

- Uses SQLite in shared in-memory mode (cache=shared)
- Uses threading to simulate concurrency
- Applies optimistic concurrency control using a version field
- Only updates if version matches expected
- Must return clear result logs per transaction

**Solution Expectations**

- Setup the DB only once (with schema + initial row)
- Each thread uses its own SQLite connection (check_same_thread=False)
- Run threads concurrently with .start() and .join()
- Each thread:
  - Reads quantity/version
  - Waits (time.sleep(delay))
  - Tries to update with WHERE version = ?
  - Commits if rowcount == 1, else rolls back
- Store outcome in shared dictionary (thread-safe)

**Function Signatures**

```python
def run_transaction(name: str, delay: float) -> None:
    pass

def simulate_concurrent_transactions(threads: list[threading.Thread]) -> list[dict]:
    pass

```

**Edge Case Behavior**

- If only 1 transaction is provided then always commits (no conflict) and return the dictionary list.
- If no row is found then it should handle the case via exception handling.
- If both threads succeed in read but only one commits then second must detect conflict with reason.
- If DB not initialized then transactions must fail gracefully returing -1.
- If exception occurs then they should be handled via exception handling and must rollback and report status = "error"

**Constraints**

- Use only standard Python libraries
- Each transaction must use its own SQLite connection
- Updates must only occur if the version matches the expected value
- No retries allowed. each transaction gets only one chance
- Only the inventory row with id = 1 can be modified
- All transaction results must be recorded in a shared dictionary by name

In [ ]:
# code

"""This module simulates OCC in SQLite."""

import sqlite3
import threading
import time
from typing import List, Dict

DB_URI = "file:occdb?mode=memory&cache=shared"
_transaction_results: Dict[str, Dict] = {}


def setup_db() -> sqlite3.Connection:
    """Initialize a shared in-memory SQLite database with an `inventory` table.

    Inserts a default item with quantity=10 and version=1.

    Returns:
        sqlite3.Connection: The open connection to keep the in-memory DB alive.
    """
    conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS inventory (
            id INTEGER PRIMARY KEY,
            item_name TEXT,
            quantity INTEGER,
            version INTEGER
        )
    ''')
    cursor.execute('''
        INSERT OR REPLACE INTO inventory (id, item_name, quantity, version)
        VALUES (1, 'Health Potion', 10, 1)
    ''')
    conn.commit()
    return conn


def run_transaction(name: str, delay: float) -> None:
    """Simulate a database transaction using optimistic concurrency control.

    Reads item data, waits for delay, checks version, and attempts update.
    If the version has changed, the transaction is rolled back.

    Args:
        name (str): Name of the transaction (used for tracking/logging).
        delay (float): Time to wait before attempting the update
        (simulates concurrency).
    """
    try:
        time.sleep(delay)
        conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
        cursor = conn.cursor()
        cursor.execute('BEGIN')

        try:
            cursor.execute(
                'SELECT quantity, version FROM inventory WHERE id = 1')
            row = cursor.fetchone()
        except sqlite3.OperationalError as e:
            if "no such table" in str(e):
                _transaction_results[name] = {
                    "transaction": name,
                    "status": -1,
                    "reason": "Database not initialized"
                }
                return
            raise

        if row is None:
            _transaction_results[name] = {
                "transaction": name,
                "status": "error",
                "reason": "Inventory row not found"
            }
            return

        current_quantity, current_version = row

        cursor.execute('SELECT version FROM inventory WHERE id = 1')
        latest_version = cursor.fetchone()[0]

        if latest_version != 1:
            conn.rollback()
            _transaction_results[name] = {
                "transaction": name,
                "status": "conflict",
                "reason": """Expected version={current_version},
                found version={latest_version}."""
            }
            return

        new_quantity = current_quantity - 1
        new_version = current_version + 1

        cursor.execute('''
            UPDATE inventory
            SET quantity = ?, version = ?
            WHERE id = 1
        ''', (new_quantity, new_version))

        conn.commit()
        _transaction_results[name] = {
            "transaction": name,
            "status": "committed",
            "final_quantity": new_quantity,
            "final_version": new_version
        }

    except Exception as e:
        conn.rollback()
        _transaction_results[name] = {
            "transaction": name,
            "status": "error",
            "reason": str(e)
        }
    finally:
        conn.close()


def simulate_concurrent_transactions(
        threads: List[threading.Thread]) -> List[Dict]:
    """Launch and manages concurrent execution of.

    provided transaction threads.
    Collects and returns the outcome of each transaction.

    Args:
        threads (List[threading.Thread]):
        A list of pre-configured transaction threads.

    Returns:
        List[Dict]: A list of transaction result dictionaries.
    """
    global _transaction_results
    _transaction_results = {}

    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    return list(_transaction_results.values())


if __name__ == "__main__":
    keep_alive_conn = setup_db()

    threads = [
        threading.Thread(target=run_transaction, args=("Transaction A", 0.3)),
        threading.Thread(target=run_transaction, args=("Transaction B", 0.5))
    ]

    results = simulate_concurrent_transactions(threads)

    keep_alive_conn.close()

    for result in results:
        print(result)


In [ ]:
# tests
"""Unit tests for SQLite OCC simulation using threading."""

import unittest
import threading
import sqlite3
from main import (
    setup_db,
    run_transaction,
    simulate_concurrent_transactions,
    DB_URI,
    _transaction_results,
)


class TestOCCSimulation(unittest.TestCase):
    """Tests for SQLite OCC simulation."""

    def setUp(self):
        """Initialize shared in-memory DB before each test."""
        self.keep_alive_conn = setup_db()

    def tearDown(self):
        """Close the DB connection after each test."""
        self.keep_alive_conn.close()

    def test_single_transaction_commits(self):
        """Test that a single transaction commits successfully."""
        thread = threading.Thread(target=run_transaction, args=("T1", 0))
        results = simulate_concurrent_transactions([thread])
        self.assertEqual(results[0]["status"], "committed")
        self.assertEqual(results[0]["final_quantity"], 9)
        self.assertEqual(results[0]["final_version"], 2)

    def test_two_transactions_conflict(self):
        """Test that two transactions cause a conflict."""
        t1 = threading.Thread(target=run_transaction, args=("T1", 0))
        t2 = threading.Thread(target=run_transaction, args=("T2", 0.1))
        results = simulate_concurrent_transactions([t1, t2])
        statuses = {r["transaction"]: r["status"] for r in results}
        self.assertIn("committed", statuses.values())
        self.assertIn("conflict", statuses.values())

    def test_three_conflicting_transactions(self):
        """Only one of three conflicting transactions should commit."""
        threads = [
            threading.Thread(target=run_transaction, args=(f"T{i}", i * 0.1))
            for i in range(3)
        ]
        results = simulate_concurrent_transactions(threads)
        committed = sum(1 for r in results if r["status"] == "committed")
        self.assertEqual(committed, 1)
        self.assertEqual(len(results), 3)

    def test_conflict_has_reason(self):
        """Ensure conflict result contains a 'reason' field."""
        t1 = threading.Thread(target=run_transaction, args=("T1", 0))
        t2 = threading.Thread(target=run_transaction, args=("T2", 0.05))
        results = simulate_concurrent_transactions([t1, t2])
        for result in results:
            if result["status"] == "conflict":
                self.assertIn("reason", result)

    def test_inventory_row_absence(self):
        """Test transaction error when inventory row is missing."""
        conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
        conn.execute("DELETE FROM inventory WHERE id=1")
        conn.commit()
        conn.close()

        t = threading.Thread(target=run_transaction, args=("T1", 0))
        results = simulate_concurrent_transactions([t])
        self.assertEqual(results[0]["status"], "error")
        self.assertIn("Inventory row not found", results[0]["reason"])

    def test_uninitialized_db_error(self):
        """Test error when database schema is missing."""
        conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
        conn.execute("DROP TABLE inventory")
        conn.commit()
        conn.close()

        t = threading.Thread(target=run_transaction, args=("T1", 0))
        results = simulate_concurrent_transactions([t])
        self.assertEqual(results[0]["status"], -1)
        self.assertIn("Database not initialized", results[0]["reason"])

    def test_transaction_delay_effect(self):
        """Test conflict behavior with delayed transaction."""
        t1 = threading.Thread(target=run_transaction, args=("Fast", 0))
        t2 = threading.Thread(target=run_transaction, args=("Slow", 0.3))
        results = simulate_concurrent_transactions([t1, t2])
        statuses = {r["transaction"]: r["status"] for r in results}
        self.assertEqual(statuses["Fast"], "committed")
        self.assertEqual(statuses["Slow"], "conflict")

    def test_exact_version_conflict(self):
        """Test conflict when version is explicitly bumped."""
        conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
        conn.execute("UPDATE inventory SET version = 2 WHERE id = 1")
        conn.commit()
        conn.close()

        t = threading.Thread(target=run_transaction, args=("T1", 0))
        results = simulate_concurrent_transactions([t])
        self.assertEqual(results[0]["status"], "conflict")

    def test_zero_delay_multiple_threads(self):
        """Only one thread should commit with zero delay."""
        threads = [
            threading.Thread(target=run_transaction, args=(f"T{i}", 0))
            for i in range(4)
        ]
        results = simulate_concurrent_transactions(threads)
        committed = sum(1 for r in results if r["status"] == "committed")
        self.assertEqual(committed, 1)

    def test_long_delay_leads_to_conflict(self):
        """Test that long delay causes a version conflict."""
        t1 = threading.Thread(target=run_transaction, args=("T1", 0))
        t2 = threading.Thread(target=run_transaction, args=("T2", 2))
        results = simulate_concurrent_transactions([t1, t2])
        self.assertIn("conflict", {r["status"] for r in results})

    def test_exception_handling(self):
        """Test that transaction exceptions are captured in results."""
        _transaction_results.clear()

        def failing_tx(name, delay):
            try:
                raise RuntimeError("forced error")
            except Exception as e:
                _transaction_results[name] = {
                    "transaction": name,
                    "status": "error",
                    "reason": str(e)
                }

        thread = threading.Thread(target=failing_tx, args=("FaultyTx", 0))
        thread.start()
        thread.join()

        results = list(_transaction_results.values())
        self.assertEqual(len(results), 1)
        result = results[0]
        self.assertEqual(result["status"], "error")
        self.assertIn("forced error", result["reason"])

    def test_transaction_keys_present(self):
        """Test that required keys are present in result."""
        t = threading.Thread(target=run_transaction, args=("T1", 0))
        results = simulate_concurrent_transactions([t])
        keys = results[0].keys()
        self.assertIn("transaction", keys)
        self.assertIn("status", keys)

    def test_quantity_never_negative(self):
        """Ensure quantity never becomes negative after commits."""
        threads = [
            threading.Thread(target=run_transaction, args=(f"T{i}", i * 0.2))
            for i in range(10)
        ]
        results = simulate_concurrent_transactions(threads)
        committed = [r for r in results if r["status"] == "committed"]
        if committed:
            min_quantity = min(r["final_quantity"] for r in committed)
            self.assertGreaterEqual(min_quantity, 0)

    def test_final_version_progression(self):
        """Ensure committed versions progress monotonically."""
        threads = [
            threading.Thread(target=run_transaction, args=(f"T{i}", i * 0.2))
            for i in range(4)
        ]
        results = simulate_concurrent_transactions(threads)
        committed_versions = sorted([
            r["final_version"] for r in results if r["status"] == "committed"
        ])
        for i in range(len(committed_versions) - 1):
            self.assertLess(committed_versions[i], committed_versions[i + 1])

    def test_transaction_naming(self):
        """Test Unicode transaction naming."""
        t = threading.Thread(target=run_transaction, args=("🧪", 0))
        results = simulate_concurrent_transactions([t])
        self.assertEqual(results[0]["transaction"], "🧪")

    def test_transaction_count_matches(self):
        """Ensure result count matches thread count."""
        n = 5
        threads = [
            threading.Thread(target=run_transaction, args=(f"T{i}", i * 0.1))
            for i in range(n)
        ]
        results = simulate_concurrent_transactions(threads)
        self.assertEqual(len(results), n)

    def test_db_stability_post_conflicts(self):
        """Test that DB remains stable after conflict."""
        t1 = threading.Thread(target=run_transaction, args=("T1", 0))
        t2 = threading.Thread(target=run_transaction, args=("T2", 0.1))
        simulate_concurrent_transactions([t1, t2])
        conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
        quantity = conn.execute(
            "SELECT quantity FROM inventory WHERE id=1"
        ).fetchone()[0]
        self.assertGreaterEqual(quantity, 0)
        conn.close()

    def test_final_version_expected(self):
        """Test that final version is incremented."""
        thread = threading.Thread(target=run_transaction, args=("VCheck", 0))
        simulate_concurrent_transactions([thread])
        conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
        version = conn.execute(
            "SELECT version FROM inventory WHERE id=1"
        ).fetchone()[0]
        self.assertGreaterEqual(version, 2)
        conn.close()

    def test_multiple_delays_affect_order(self):
        """Only one transaction should commit due to spacing."""
        delays = [0.4, 0.1, 0.2]
        names = [f"T{i}" for i in range(3)]
        threads = [
            threading.Thread(
                target=run_transaction, args=(names[i], delays[i])
            ) for i in range(3)
        ]
        results = simulate_concurrent_transactions(threads)
        committed = [r for r in results if r["status"] == "committed"]
        self.assertEqual(len(committed), 1)


ModuleNotFoundError: No module named 'main'

# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/eb517933-3043-4b37-a0cd-7c333a713e9d?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67701%2Fview

#### Model code:

```python
import sqlite3
import threading
import time
from typing import List, Dict

# DB URI for shared, in-memory SQLite database
DB_URI = "file:occdb?mode=memory&cache=shared"

# Initializes the SQLite database
def setup_db():
    # Connect to the database
    conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
    cursor = conn.cursor()

    # Create the inventory table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS inventory (
            id INTEGER PRIMARY KEY,
            item_name TEXT,
            quantity INTEGER,
            version INTEGER
        )
    ''')

    # Insert initial data into the table
    cursor.execute('''
        INSERT INTO inventory (id, item_name, quantity, version)
        VALUES (1, 'Health Potion', 10, 1)
    ''')

    # Commit changes and close the connection
    conn.commit()
    conn.close()

# <Issue>: <Wrong Function Signature: 'results' was not supposed to be a parameter here.>

# Function to simulate a transaction
def run_transaction(name: str, delay: float, results: Dict[str, Dict]):
    try:
        conn = sqlite3.connect(DB_URI, uri=True, check_same_thread=False)
        cursor = conn.cursor()

# <Issue>: <Does not check if the database exists that was mentioned in edge case behavior of prompt>

        # Start transaction
        cursor.execute('BEGIN')

        # Read current inventory quantity and version
        cursor.execute('SELECT quantity, version FROM inventory WHERE id = 1')
        row = cursor.fetchone()

        if row is None:
            raise Exception("No inventory row found.")

        current_quantity, current_version = row

        # Simulate transaction delay
        time.sleep(delay)

        # Try to update the record with optimistic concurrency control
        new_quantity = current_quantity - 1
        new_version = current_version + 1
        cursor.execute('''
            UPDATE inventory
            SET quantity = ?, version = ?
            WHERE id = 1 AND version = ?
        ''', (new_quantity, new_version, current_version))

        # Check if update was successful

# <Issue>: <Wrong Logic used in if condition>

        if cursor.rowcount == 1:
            conn.commit()
            results[name] = {
                "transaction": name,
                "status": "committed",
                "final_quantity": new_quantity,
                "final_version": new_version
            }
        else:
            conn.rollback()
            results[name] = {
                "transaction": name,
                "status": "conflict",
                "reason": f"Expected version={current_version}, found different."
            }

    except Exception as e:
        conn.rollback()
        results[name] = {
            "transaction": name,
            "status": "error",
            "reason": str(e)
        }
    finally:
        conn.close()

# Function to simulate concurrent transactions
def simulate_concurrent_transactions(threads: List[threading.Thread]) -> List[Dict]:

# <Issue>: <db should be initialzed in main to avoid bugs during multiple input>
    # Initialize shared database with schema and initial data
    setup_db()

    # Dictionary to store transaction results
    results = {}

    # Start all threads
    for t in threads:
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()

    # Convert results dictionary to a list
    return list(results.values())

# Example usage
if __name__ == "__main__":

# <Issue>: <Raises NameError: results is used before being defined>
# <Issue>: <Wrong Input used. no 'result' parameter was mentioned in Example>

    threads = [
        threading.Thread(target=run_transaction, args=("Transaction A", 0.3, results)),
        threading.Thread(target=run_transaction, args=("Transaction B", 0.5, results)),
    ]
    results = simulate_concurrent_transactions(threads)
    for result in results:
        print(result)
```